In [1]:
import torch
import random

random_seed = 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(random_seed)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(random_seed)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(random_seed)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Using device: cpu


In [2]:
from pan21_functions import *
import datetime
from keras.callbacks import CSVLogger
from pathlib import Path

In [3]:
import importlib; import pan21_functions as p21; importlib.reload(p21)

<module 'pan21_functions' from 'c:\\Users\\thoma\\Documents\\croatia\\masters\\semester2\\text_analysis\\tar_project\\pan21_functions.py'>

In [4]:
# Pan21PyDataset("pan21/train", "pan21/train", "train_ds_uncompressed").to_file()
# Pan21PyDataset("pan21/validation", "pan21/validation", "val_ds_uncompressed").to_file()

In [5]:
# Make sure the files are good
def test_load(path, name):
    np.load(path)[name]

# _ = Parallel(n_jobs=-1)(delayed(test_load)(Path("train_ds") / f"{i}.npz", "batch_x") for i in tqdm(range(350)))

In [6]:
# import time

# start_idx = 0
# limit = 5
# # limit = len(train_ds)

# before = time.time()
# for i in range(start_idx, start_idx+limit):
#     batch_x, batch_y = train_ds.__getitem__(i, force_compute=True)
# after = time.time()
# for i in range(start_idx, start_idx+limit):
#     batch_x, batch_y = train_ds.__getitem__(i, force_compute=False)
# after_after = time.time()

# print(f"Compute: {round((after - before)/limit, 2)}s vs File read: {round((after_after - after)/limit, 2)}s")
# # Compute: 8.95s per batch
# # Compute with compression: ~17s
# # Uncompressed read: .7s per batch
# # Compressed read: 1.6s per batch
# # Compressed is 1/10 the size of uncompressed, but takes ~twice as long to precompute and save
# # Compressed 512D Fourier takes ~30s per batch
# # Compressed 512D Fourier is about 500-700MB per batch

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Input, Flatten
from tensorflow.keras import optimizers, losses, metrics

# Code implementation of the RNN for sequence labeling
def create_rnn_model(num_labels, embedding_dim, max_input_length):
    model = Sequential()
    model.add(Input(shape=(max_input_length*2, embedding_dim)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_labels, activation='sigmoid'))

    model.compile(
        optimizer=optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=losses.BinaryCrossentropy(),
        # List of metrics to monitor
        metrics=[metrics.BinaryAccuracy(), metrics.AUC()],
        jit_compile=True
    )

    return model
 
num_labels = 1
embedding_dim = 768

In [12]:
from transformers import TFBertModel, AutoTokenizer



def build_text_model():
    model_name = 'bert-base-cased'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    bert = TFBertModel.from_pretrained(model_name)
    
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    def encode_text(text):
        inputs = [tf.compat.as_str(x) for x in text.numpy().tolist()]
        tokenized = tokenizer(
            inputs,
            return_tensors='tf',
            add_special_tokens=True,
            max_length=110,
            padding='max_length',
            truncation=True)
        return tokenized['input_ids'], tokenized['attention_mask']
        
    input_ids, attention_mask = tf.py_function(encode_text, inp=[text_input], Tout=[tf.int32, tf.int32])
    
    input_ids = tf.ensure_shape(input_ids, [None, 110])
    attention_mask = tf.ensure_shape(attention_mask, [None, 110])
    
    outputs = bert(input_ids, attention_mask)
    
    net = outputs['last_hidden_state']

    # Some other layers, this part is not important
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(net)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, name='classifier'))(x)

    return tf.keras.Model(inputs=text_input, outputs=x)

In [9]:
def train_model_num_ff(train_ds, val_ds, model_name, epochs=5):
    max_input_length = train_ds.max_input_length
    # print(f"{max_input_length=}")

    print(model_name)
    time_string = f"{datetime.datetime.now().strftime('%Y_%m_%d-%I_%M_%S_%p')}"
    model_name.mkdir(exist_ok=True)
    checkpoint_name_format = time_string + "_cp-{epoch:02d}.weights.h5"
    checkpoint_path = model_name / checkpoint_name_format
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, 
        verbose=1, 
        save_weights_only=True,
        # Save weights, every epoch.
        save_freq='epoch')

    model = create_rnn_model(num_labels, embedding_dim, max_input_length)
    csv_logger = CSVLogger(f'{model_name}_{time_string}.log', separator=',', append=False)

    already_trained_epochs = 0
    if model_name.exists():
        checkpoints = list(Path(model_name).glob('*.weights.h5'))
        if checkpoints:
            already_trained_epochs = len(checkpoints)
            model.load_weights(checkpoints[-1])

    if DEVICE == "cuda":
        with tf.device("/device:GPU:0"):
            history = model.fit(
                train_ds,
                epochs=epochs - already_trained_epochs,
                validation_data=val_ds,
                verbose=1,
                callbacks=[csv_logger, cp_callback]
            )
    else:
        history = model.fit(
            train_ds,
            epochs=epochs - already_trained_epochs,
            validation_data=val_ds,
            verbose=1,
            callbacks=[csv_logger, cp_callback]
        )

    model.save(f"{model_name}.keras")

In [10]:
# Does using the frequency domain spectra provide useful information?
model_dir = Path(f"models/num_fourier_features")
if not model_dir.exists():
    model_dir.mkdir(parents=True)

num_fourier_features = [0, 512]
for num_ff in tqdm(num_fourier_features):
    fourier_train_ds = p21.Pan21FourierDataset("pan21/train", "pan21/train", num_fourier_features=num_ff)
    fourier_val_ds = p21.Pan21FourierDataset("pan21/validation", "pan21/validation", num_fourier_features=num_ff)

    model_name = model_dir / f"ff_{num_ff}"

    train_model_num_ff(fourier_train_ds, fourier_val_ds, model_name)

  0%|          | 0/2 [00:00<?, ?it/s]

models\num_fourier_features\ff_0
Epoch 1/5
1402/2186 ━━━━━━━━━━━━━━━━━━━━ 49:35 4s/step - auc: 0.5262 - binary_accuracy: 0.5487 - loss: 0.6907

c:\Users\thoma\Documents\croatia\masters\semester2\text_analysis\tar_project\.venv311\Lib\site-packages\joblib\_store_backends.py:215: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: [Errno 28] No space left on device.
  warnings.warn(


1485/2186 ━━━━━━━━━━━━━━━━━━━━ 44:08 4s/step - auc: 0.5279 - binary_accuracy: 0.5494 - loss: 0.6904

KeyboardInterrupt: 

In [ ]:
# How many epochs should be trained?
model_dir = Path(f"models/num_epochs")
if not model_dir.exists():
    model_dir.mkdir(parents=True)

num_fourier_features = [0, 512//4]
for num_ff in tqdm(num_fourier_features):
    train_model_num_ff(num_ff, epochs=15)

In [ ]:
# Does filtering help?
model_dir = Path(f"models/cutoffs")
if not model_dir.exists():
    model_dir.mkdir(parents=True)

N = 4 # High pass, band stop, band stop, low pass
# Frequencies respresented as percent of Nyquist frequency
cutoff_frequencies = [(i / N, (i + 1) / N) for i in range(N)]
for cutoff in tqdm(cutoff_frequencies):
    filter_train_ds = p21.Pan21FourierFilterDataset("pan21/train", "pan21/train", cutoff)
    filter_val_ds = p21.Pan21FourierFilterDataset("pan21/validation", "pan21/validation", cutoff)

    model_name = model_dir / f"{cutoff[0]}_{cutoff[1]}"

    train_model_num_ff(filter_train_ds, filter_val_ds, model_name, epochs = 3)